# Toronto - DS Capstone assignment - 1

### Importing libraies

In [1]:
import pandas as pd
import requests
import lxml.html as lxht

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Processing information about Toronto provided on the Wikipedia link

In [3]:
tor_raw = requests.get(link)
tor_doc = lxht.fromstring(tor_raw.content)
tor_con = tor_doc.xpath('//tr')

### Creating a List with first element

In [4]:
tor_con = tor_doc.xpath('//tr')

tor_col=[]
n=0

for k in tor_con[0]:
    n+=1
    tor_name=k.text_content()
    print ('%d:"%s"'%(n,tor_name))
    tor_col.append((tor_name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


### Creating a DataFrame

In [5]:
for k in range(1,len(tor_con)):
    tor=tor_con[k]
    if len(tor)!=3:
        break
    
    n=0
    for m in tor.iterchildren():
        tor_data=m.text_content() 
        if n>0:
            try:
                tor_data=int(tor_data)
            except:
                pass
        tor_col[n][1].append(tor_data)
        n+=1
        
tor_dict={title:column for (title,column) in tor_col}
df=pd.DataFrame(tor_dict)

In [6]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


### Cleaning the DataFrame

In [7]:
df = df.replace('\n','', regex=True)
df.columns = ['Postalcode', 'Borough', 'Neighbourhood',]
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Dropping 'Not assigned' cells from the Borough column of the DataFrame

In [8]:
df.drop(df.index[df['Borough'] == 'Not assigned'], axis = 0, inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Grouping the DataFrame by Postalcode and Borough

In [9]:
df = df.groupby(['Postalcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postalcode','Borough','Neighbourhood']
df.head()

,Postalcode,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


### Dropping 'Canadian postal codes' cell from the Borough column of the DataFrame

In [10]:
df.drop(df.index[df['Borough'] == 'Canadian postal codes'], axis = 0, inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Assigning Neighbourhood = Borough where Neighbourhood is 'Not Assigned'

In [11]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [12]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Checking shape as the final step of the assignment

In [13]:
df.shape

(103, 3)